In [ ]:
! pip install transformers datasets sacrebleu rouge_score py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 12.1 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset, load_metric
import pandas as pd
import tqdm
import numpy as np

In [ ]:
dataset_samsum = load_dataset('samsum', split = "train")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


In [ ]:
dataset_samsum = dataset_samsum.train_test_split(test_size=0.2)

In [ ]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 11785
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 2947
    })
})

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
tokenized_samsum = dataset_samsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/11785 [00:00<?, ? examples/s]

Map:   0%|          | 0/2947 [00:00<?, ? examples/s]

In [ ]:
rouge_metric = load_metric("rouge")

<ipython-input-16-46a4432edece>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.3 MB/s eta 0:00:00


In [ ]:
import evaluate

rouge = evaluate.load("rouge")
def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

In [ ]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
from transformers import TrainingArguments, Trainer


training_args = Seq2SeqTrainingArguments(

    output_dir='T5-base-samsum',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    # fp16=True,
    # push_to_hub=True,

)

NameError: ignored

In [ ]:
trainer = Seq2SeqTrainer(
                  model=model, 
                  args=training_args,
                  tokenizer=tokenizer, 
                  data_collator=data_collator,
                  train_dataset=tokenized_samsum["train"], 
                  eval_dataset=tokenized_samsum["test"],
                  compute_metrics=compute_metrics)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.035400,1.872434,0.395900,0.165800,0.325400,0.325400,16.280600
2,1.958700,1.831596,0.405600,0.175600,0.334100,0.334100,16.522600
3,1.931800,1.814938,0.410200,0.179400,0.338900,0.338800,16.573100
4,1.912600,1.808800,0.409600,0.178900,0.339100,0.339100,16.581900


TrainOutput(global_step=11788, training_loss=1.974330584963257, metrics={'train_runtime': 2121.6581, 'train_samples_per_second': 22.218, 'train_steps_per_second': 5.556, 'total_flos': 3364999509245952.0, 'train_loss': 1.974330584963257, 'epoch': 4.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
     

Mounted at /content/drive


In [ ]:
## Save model
model.save_pretrained("/content/drive/MyDrive/Saved models/T5-samsum-model")

In [ ]:
## Save tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Saved models/T5-tokenizer") 

('/content/drive/MyDrive/Saved models/T5-tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Saved models/T5-tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Saved models/T5-tokenizer/tokenizer.json')